# Data Collection

To collect data for whether the robot should grip or loose the feeding bag, images are collected for whether the bowl is full or empty and the jetson will base its decisions upon this. 

1. Get the jetson to take pictures of bowls filled with food and label these ``full``. 

2. Get the jetson to take pictures of empty bowls and label these ``empty``. 

By varying bowls, lighting, types and amount of food, the more accurate it will be.

### Showing live camera feed

In [1]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

# If all works, the image will be visible below
display(image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

### Creating the folder ``dataset`` with two sub-folders ``full`` and ``empty`` 

In [2]:
import os

full_dir = 'dataset/full'
empty_dir = 'dataset/empty'

# If the directories already exists 
try:
    os.makedirs(full_dir)
    os.makedirs(empty_dir)
except FileExistsError:
    print('Directories already exist')

Directories already exist


### Creating buttons for data collection

In [3]:
button_layout = widgets.Layout(width='128px', height='64px')
full_button = widgets.Button(description='add full', button_style='info', layout=button_layout)
empty_button = widgets.Button(description='add empty', button_style='success', layout=button_layout)
full_count = widgets.IntText(layout=button_layout, value=len(os.listdir(full_dir)))
empty_count = widgets.IntText(layout=button_layout, value=len(os.listdir(empty_dir)))

### Connecting buttons to saving functions

In [7]:
from uuid import uuid1

def save_image(directory):
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)
    
def save_full():
    global full_dir, full_count
    save_image(full_dir)
    full_count.value = len(os.listdir(full_dir))
    
def save_empty():
    global empty_dir, empty_count
    save_image(empty_dir)
    empty_count.value = len(os.listdir(empty_dir))

empty_button.on_click(lambda x: save_empty())
full_button.on_click(lambda x: save_full())

### Collect images

In [8]:
display(image)
display(widgets.HBox([empty_count, empty_button]))
display(widgets.HBox([full_count, full_button]))

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Closing the camera

In [9]:
camera.stop()

## Zipping the dataset

In [10]:
!zip -r -q dataset.zip dataset

### Next: train the model